### 크롤링으로  업종별 코드 가져오기

In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import numpy as np
import pandas as pd

import time
import urllib.request
from selenium.webdriver import Chrome
import json
import re
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import datetime as dt

#print(upjong_code)

In [2]:
# 업종코드를 입력하면 --> 해당 기업 코드가 나오는 함수. 
def upjong_url(code):
    tickers = {}
    url_tmp = "https://finance.naver.com/sise/sise_group_detail.nhn?type=upjong&no=%s"
    url = url_tmp % (code)
    res = requests.get(url)
    html = res.text
    
    soup = BeautifulSoup(html,"html.parser")
    rows = soup.select('td > div')

    for r in rows[1:]:
        tickers[r.select('a')[0]['href'][20:]] = r.text
    a = [k for k in tickers.keys() if tickers[k][-2]=='우' or tickers[k][-3]=='우']
    # 우량주 종목 제외    
    for i in range(len(a)):
        del tickers[a[i]]

    code = list(tickers.keys())
    com = list(tickers.values())
    return code,com
#회사 코드를 사용해서 url완성하는 함수
def ttt(xx):
    name = xx
    base_url = 'https://finance.naver.com/item/coinfo.nhn?code='+ name + '&target=finsum_more'
    return base_url
# 종목코드를 통해 재무제표 크롤링
def stock_crawler(code):
    #code = 종목번호
    name = code
    base_url = 'https://finance.naver.com/item/coinfo.nhn?code='+ name + '&target=finsum_more'
    print(base_url, '브라우저 겟 시작')
    browser.get(base_url)
    print(base_url, '브라우저 겟 종료')
    #frmae구조 안에 필요한 데이터가 있기 때문에 해당 데이터를 수집하기 위해서는 frame구조에 들어가야한다.
    browser.switch_to.frame(browser.find_element_by_id('coinfo_cp'))
    
    #재무제표 "연간" 클릭하기
    browser.find_elements_by_xpath('//*[@class="schtab"][1]/tbody/tr/td[3]')[0].click()

    html0 = browser.page_source
    html1 = BeautifulSoup(html0,'html.parser')
    
    #기업명 뽑기
    title0 = html1.find('head').find('title').text
    #print(title0.split('-')[-1])
    
    html22 = html1.find('table',{'class':'gHead01 all-width','summary':'주요재무정보를 제공합니다.'})
    
    #date scrapy
    thead0 = html22.find('thead')
    tr0 = thead0.find_all('tr')[1]
    th0 = tr0.find_all('th')
    
    date = []
    for i in range(len(th0)):
        date.append(''.join(re.findall('[0-9/]',th0[i].text)))
    
    #columns scrapy
    tbody0 = html22.find('tbody')
    tr0 = tbody0.find_all('tr')
    
    col = []
    for i in range(len(tr0)):

        if '\xa0' in tr0[i].find('th').text:
            tx = re.sub('\xa0','',tr0[i].find('th').text)
        else:
            tx = tr0[i].find('th').text

        col.append(tx)
    
    #main text scrapy
    td = []
    for i in range(len(tr0)):
        td0 = tr0[i].find_all('td')
        td1 = []
        for j in range(len(td0)):
            if td0[j].text == '':
                td1.append('0')
            else:
                td1.append(td0[j].text)

        td.append(td1)
    
    td2 = list(map(list,zip(*td)))
    
    return pd.DataFrame(td2,columns = col,index = date).T
# numpy -> pandas 데이터프레임으로 변환하는 사용자 정의 함수
def trans_df(ar,com):
    #df = np.array([])
    metrics = ['PER(배)','PBR(배)','EPS(원)','ROE(%)','ROA(%)','당기순이익']
    ar = np.resize(ar,(int(len(com)),6))
    #ar.resize((int(len(com)),6))
    df = pd.DataFrame(data = ar,index=com,columns = metrics)
    return df

In [3]:

# selenium을 위한 chrome 브라우저 켜기
browser  = Chrome()
#browser  = IE()
browser.maximize_window()

annual_date = ['2015/12', '2016/12', '2017/12', '2018/12', '2019/12', '2020/12','2021/12', '2022/12']
# 각각 지표들 데이터프레임에 넣기 위한 초기 선언, (df_NI(Net Income) : 당기순이익을 뜻함)

url_up = 'https://finance.naver.com/sise/sise_group.nhn?type=upjong'

## 밑에는 업종별 코드를 반환하기 위한 코드 
res = requests.get(url_up)
html = res.text
soup = BeautifulSoup(html,'html.parser')

upjong = {}
rows = soup.select('td > a')

for r in range(len(rows)):
    upjong[soup.select('td > a')[r]['href'][43:]] = soup.select('td > a')[r].text
upjong_code = list(upjong.keys())[:-1]
# 기타 업종 제외 -> 재무제표 제공 X 
del upjong_code[45]

tickers = {}
for up in upjong_code:
    code, com = upjong_url(up)
    for i in range(len(code)):
        tickers[code[i]] = com[i]
        
# res가 업종별 5순위 종목을 누적해서 데이터프레임에 저장 업종코드 res라는 전혀 다른 의미.

res = pd.DataFrame([])
com,toplist = [],[]
# 지표 기준 년도 
#sc ='2019/12'
p = re.compile('2019.[0-9][0-9]')
# 지표 기준 년도 
#sc ='2019/12'
i = 1
for up in upjong_code[:3]:
    # 업종별로 회사 코드를 받음, 전자장비및기기업종 -> 78개 회사 리스트를 받는다.
    # 몇 번째 업종 불러오는지 표시하기 위한 것으로 주석처리해도 무방
    print('진행상황 : {}/{}'.format(i,len(upjong_code)))
    df_com = np.array([])
    code, _ = upjong_url(up)
    for c in code:
        # 재무제표를 종목별로 불러와 지표 저장
        df = stock_crawler(c)
        df = df.apply(pd.to_numeric,errors='coerce')
        for d in list(df.columns):
            if p.findall(d)!= []:
                sc = p.findall(d)[0]
        if df.loc[['PER(배)','PBR(배)','EPS(원)','ROE(%)','ROA(%)','당기순이익'],sc].isnull().sum() == 0 & (df.sum().sum()>0):
            df_com = np.hstack([df_com,df.loc[['PER(배)','PBR(배)','EPS(원)','ROE(%)','ROA(%)','당기순이익'],sc].values])
            com.append(tickers[c])
    # 데이터프레임으로 변환
    df_com = trans_df(df_com,com)
    # 지표별로 랭크 구하여 합산하기
    df_com['upjong_rank'] = (df_com[['PER(배)','PBR(배)']].rank().sum(axis=1)+df_com[['ROE(%)','ROA(%)','EPS(원)','당기순이익']].rank(method='max').sum(axis=1)).rank()
    df_com.sort_values(by='upjong_rank',inplace=True)
    #toplist.append(list(df_com[df_com['upjong_rank']<=5].index))
    res = pd.concat([res,df_com[df_com['upjong_rank']<=5]])
    i+=1
res['total_rank'] = (res[['PER(배)','PBR(배)']].rank().sum(axis=1)+res[['ROE(%)','ROA(%)','EPS(원)','당기순이익']].rank(method='max').sum(axis=1)).rank()
res.sort_values(by='total_rank',inplace=True)

진행상황 : 1/78
https://finance.naver.com/item/coinfo.nhn?code=034020&target=finsum_more 브라우저 겟 시작
https://finance.naver.com/item/coinfo.nhn?code=034020&target=finsum_more 브라우저 겟 종료
https://finance.naver.com/item/coinfo.nhn?code=297090&target=finsum_more 브라우저 겟 시작
https://finance.naver.com/item/coinfo.nhn?code=297090&target=finsum_more 브라우저 겟 종료
https://finance.naver.com/item/coinfo.nhn?code=013810&target=finsum_more 브라우저 겟 시작
https://finance.naver.com/item/coinfo.nhn?code=013810&target=finsum_more 브라우저 겟 종료
https://finance.naver.com/item/coinfo.nhn?code=044490&target=finsum_more 브라우저 겟 시작
https://finance.naver.com/item/coinfo.nhn?code=044490&target=finsum_more 브라우저 겟 종료
https://finance.naver.com/item/coinfo.nhn?code=101170&target=finsum_more 브라우저 겟 시작
https://finance.naver.com/item/coinfo.nhn?code=101170&target=finsum_more 브라우저 겟 종료
https://finance.naver.com/item/coinfo.nhn?code=100130&target=finsum_more 브라우저 겟 시작
https://finance.naver.com/item/coinfo.nhn?code=100130&target=finsum_more 브라

KeyboardInterrupt: 

In [ ]:
res

In [10]:
# res가 최종 데이터프레임, 회사명, 지표별, 랭크 순위
# upjong_rank는 업종내에서 1~5순위 랭크 total_rank는 모든 종목에 대해서 매긴 랭크 순위
res.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>PER(배)</th>\n      <th>PBR(배)</th>\n      <th>EPS(원)</th>\n      <th>ROE(%)</th>\n      <th>ROA(%)</th>\n      <th>당기순이익</th>\n      <th>upjong_rank</th>\n      <th>total_rank</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>한신기계</th>\n      <td>152.31</td>\n      <td>0.71</td>\n      <td>28.0</td>\n      <td>0.50</td>\n      <td>0.43</td>\n      <td>5.0</td>\n      <td>4.0</td>\n      <td>1.5</td>\n    </tr>\n    <tr>\n      <th>서암기계공업 *</th>\n      <td>939.72</td>\n      <td>1.30</td>\n      <td>3.0</td>\n      <td>0.15</td>\n      <td>0.08</td>\n      <td>1.0</td>\n      <td>5.0</td>\n      <td>1.5</td>\n    </tr>\n    <tr>\n      <th>혜인</th>\n      <td>171.77</td>\n      <td>0.73</td>\n      <td>44.0</td>\n      <td>0.42</td>\n      <td>0.07</td>\n      <td>6.0</td>\n      <td>3.0</td>\n      <td>3.0</td>\n    </tr>\n    <tr>\n      <th>에이치케이 *</th>\n      <t

In [13]:

from django.shortcuts import render
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
#from webdriver_manager.chrome import ChromeDriverManager
from sklearn.datasets import load_iris
from django.http import HttpResponse
#[/root/.wdm/drivers/chromedriver/linux64/85.0.4183.87/chromedriver]
from datetime import datetime
import time
import urllib.request
from selenium.webdriver import Chrome
import json
import re
from selenium.webdriver.common.keys import Keys
import datetime as dt

# 업종코드를 입력하면 --> 해당 기업 코드가 나오는 함수.
def upjong_url(code):
    tickers = {}
    url_tmp = "https://finance.naver.com/sise/sise_group_detail.nhn?type=upjong&no=%s"
    url = url_tmp % (code)
    res = requests.get(url)
    html = res.text

    soup = BeautifulSoup(html,"html.parser")
    rows = soup.select('td > div')

    for r in rows[1:]:
        tickers[r.select('a')[0]['href'][20:]] = r.text
    a = [k for k in tickers.keys() if tickers[k][-2]=='우' or tickers[k][-3]=='우']
    # 우량주 종목 제외
    for i in range(len(a)):
        del tickers[a[i]]

    code = list(tickers.keys())
    com = list(tickers.values())
    return code,com
#회사 코드를 사용해서 url완성하는 함수
def ttt(xx):
    name = xx
    base_url = 'https://finance.naver.com/item/coinfo.nhn?code='+ name + '&target=finsum_more'
    return base_url

def stock_crawler(code):
    #code = 종목번호
    name = code
    base_url = 'https://finance.naver.com/item/coinfo.nhn?code='+ name + '&target=finsum_more'
    #print(base_url, '브라우저 겟 시작')
    browser.get(base_url)
    #print(base_url, '브라우저 겟 종료')
    #frmae구조 안에 필요한 데이터가 있기 때문에 해당 데이터를 수집하기 위해서는 frame구조에 들어가야한다.
    browser.switch_to.frame(browser.find_element_by_id('coinfo_cp'))

    #재무제표 "연간" 클릭하기
    browser.find_elements_by_xpath('//*[@class="schtab"][1]/tbody/tr/td[3]')[0].click()

    html0 = browser.page_source
    html1 = BeautifulSoup(html0,'html.parser')

    #기업명 뽑기
    title0 = html1.find('head').find('title').text

    print(title0.split('-')[-1]) ####################

    html22 = html1.find('table',{'class':'gHead01 all-width','summary':'주요재무정보를 제공합니다.'})

    #date scrapy
    thead0 = html22.find('thead')
    tr0 = thead0.find_all('tr')[1]
    th0 = tr0.find_all('th')

    date = []
    for i in range(len(th0)):
        date.append(''.join(re.findall('[0-9/]',th0[i].text)))

    #columns scrapy
    tbody0 = html22.find('tbody')
    tr0 = tbody0.find_all('tr')

    col = []
    for i in range(len(tr0)):

        if '\xa0' in tr0[i].find('th').text:
            tx = re.sub('\xa0','',tr0[i].find('th').text)
        else:
            tx = tr0[i].find('th').text

        col.append(tx)

    #main text scrapy
    td = []
    for i in range(len(tr0)):
        td0 = tr0[i].find_all('td')
        td1 = []
        for j in range(len(td0)):
            if td0[j].text == '':
                td1.append('0')
            else:
                td1.append(td0[j].text)

        td.append(td1)

    td2 = list(map(list,zip(*td)))

    return pd.DataFrame(td2,columns = col,index = date).T
# numpy -> pandas 데이터프레임으로 변환하는 사용자 정의 함수
def trans_df(ar,com):
    #df = np.array([])
    metrics = ['PER(배)','PBR(배)','EPS(원)','ROE(%)','ROA(%)','당기순이익']
    ar = np.resize(ar,(int(len(com)),6))
    #ar.resize((int(len(com)),6))
    df = pd.DataFrame(data = ar,index=com,columns = metrics)
    return df

# 리눅스에서 셀레니움을 위한 환경
# options = Options()
# options.add_argument("--headless")
# options.add_argument("--no-sandbox")
# browser = webdriver.Chrome(options=options)
# WINDOW_SIZE = "1920,1080"
# options.add_argument("--window-size=%s" % WINDOW_SIZE)

browser  = Chrome()
browser.maximize_window()

annual_date = ['2015/12', '2016/12', '2017/12', '2018/12', '2019/12', '2020/12','2021/12', '2022/12']
# 각각 지표들 데이터프레임에 넣기 위한 초기 선언, (df_NI(Net Income) : 당기순이익을 뜻함)

url_up = 'https://finance.naver.com/sise/sise_group.nhn?type=upjong'

## 밑에는 업종별 코드를 반환하기 위한 코드
res = requests.get(url_up)
html = res.text
soup = BeautifulSoup(html,'html.parser')

upjong = {}
rows = soup.select('td > a')

for r in range(len(rows)):
    upjong[soup.select('td > a')[r]['href'][43:]] = soup.select('td > a')[r].text
upjong_code = list(upjong.keys())[:-1]
# 기타 업종 제외 -> 재무제표 제공 X
del upjong_code[45]

tickers = {}
for up in upjong_code:
    code, com = upjong_url(up)
    for i in range(len(code)):
        tickers[code[i]] = com[i]

# res가 업종별 5순위 종목을 누적해서 데이터프레임에 저장 업종코드 res라는 전혀 다른 의미.

res = pd.DataFrame([])
com,toplist = [],[]
# 지표 기준 년도
#sc ='2019/12'
p = re.compile('2019.[0-9][0-9]')
# 지표 기준 년도
#sc ='2019/12'
i = 1
for up in upjong_code[:3]: # 테스트로 3개 기업만.
    # 업종별로 회사 코드를 받음, 전자장비및기기업종 -> 78개 회사 리스트를 받는다.
    # 몇 번째 업종 불러오는지 표시하기 위한 것으로 주석처리해도 무방
    print('진행상황 : {}/{}'.format(i,len(upjong_code)))
    df_com = np.array([])
    code, _ = upjong_url(up)
    for c in code:
        # 재무제표를 종목별로 불러와 지표 저장
        df = stock_crawler(c)
        #print(df, 'stock_crawler입니다. \n')
        df = df.apply(pd.to_numeric,errors='coerce')
        for d in list(df.columns):
            if p.findall(d)!= []:
                sc = p.findall(d)[0]
        if df.loc[['PER(배)','PBR(배)','EPS(원)','ROE(%)','ROA(%)','당기순이익'],sc].isnull().sum() == 0 & (df.sum().sum()>0):
            df_com = np.hstack([df_com,df.loc[['PER(배)','PBR(배)','EPS(원)','ROE(%)','ROA(%)','당기순이익'],sc].values])
            com.append(tickers[c])
    # 데이터프레임으로 변환
    df_com = trans_df(df_com,com)
    # 지표별로 랭크 구하여 합산하기
    df_com['upjong_rank'] = (df_com[['PER(배)','PBR(배)']].rank().sum(axis=1)+df_com[['ROE(%)','ROA(%)','EPS(원)','당기순이익']].rank(method='max').sum(axis=1)).rank()
    df_com.sort_values(by='upjong_rank',inplace=True)
    #toplist.append(list(df_com[df_com['upjong_rank']<=5].index))
    res = pd.concat([res,df_com[df_com['upjong_rank']<=5]])
    i+=1
res['total_rank'] = (res[['PER(배)','PBR(배)']].rank().sum(axis=1)+res[['ROE(%)','ROA(%)','EPS(원)','당기순이익']].rank(method='max').sum(axis=1)).rank()
res.sort_values(by='total_rank',inplace=True)

# res가 최종 데이터프레임, 회사명, 지표별, 랭크 순위
# upjong_rank는 업종내에서 1~5순위 랭크 total_rank는 모든 종목에 대해서 매긴 랭크 순위

res

진행상황 : 1/78
 기업개요(한익스프레스)
 기업개요(동방)
 기업개요(현대글로비스)
 기업개요(KCTC)
 기업개요(삼일)
 기업개요(한솔로지스틱스)
 기업개요(인터지스)
 기업개요(유수홀딩스)
 기업개요(국보)
 기업개요(세방)
 기업개요(CJ대한통운)
 기업개요(한진)
진행상황 : 2/78
 기업개요(두산중공업)
 기업개요(맥스로텍)
 기업개요(스페코)
 기업개요(씨에스베어링)
 기업개요(씨에스윈드)
 기업개요(한국테크놀로지)
 기업개요(동양물산)
 기업개요(비디아이)
 기업개요(태웅)
 기업개요(비엠티)
 기업개요(동국S&C)
 기업개요(부스타)
 기업개요(두산인프라코어)
 기업개요(이더블유케이)
 기업개요(에스피시스템스)
 기업개요(서암기계공업)
 기업개요(엔케이)
 기업개요(한신기계)
 기업개요(자비스)
 기업개요(뉴로스)
 기업개요(현대건설기계)
 기업개요(우진플라임)
 기업개요(흥국)
 기업개요(대창솔루션)
 기업개요(에쎈테크)
 기업개요(파라텍)
 기업개요(대동기어)
 기업개요(케이피에프)
 기업개요(포메탈)
 기업개요(수산중공업)
 기업개요(디케이락)
 기업개요(한국정밀기계)
 기업개요(아세아텍)
 기업개요(알에스오토메이션)
 기업개요(현대로템)
 기업개요(디와이)
 기업개요(이엠코리아)
 기업개요(KC코트렐)
 기업개요(디와이파워)
 기업개요(SIMPAC)
 기업개요(우림기계)
 기업개요(에이테크솔루션)
 기업개요(현대엘리베이)
 기업개요(비에이치아이)
 기업개요(대동공업)
 기업개요(프리엠스)
 기업개요(로보스타)
 기업개요(삼영엠텍)
 기업개요(에이치케이)
 기업개요(휴림로봇)
 기업개요(코윈테크)
 기업개요(TPC)
 기업개요(조광ILI)
 기업개요(두산밥캣)
 기업개요(S&TC)
 기업개요(화성밸브)
 기업개요(태광)
 기업개요(웰크론한텍)
 기업개요(혜인)
 기업개요(대창단조)
 기업개요(성광벤드)
 기업개요(에버다임)
 기업개요(넥스턴)
 기업개요(영풍정밀)
 기업개요(동일금속)
 기업개요(디에스티)
 기업개요(코메론)
 기업

,PER(배),PBR(배),EPS(원),ROE(%),ROA(%),당기순이익,upjong_rank,total_rank
한신기계,152.31,0.71,28.0,0.50,0.43,5.0,4.0,1.5
서암기계공업 *,939.72,1.30,3.0,0.15,0.08,1.0,5.0,1.5
혜인,171.77,0.73,44.0,0.42,0.07,6.0,3.0,3.0
에이치케이 *,68.41,0.42,65.0,0.61,0.42,12.0,1.0,4.0
유수홀딩스,42.59,0.73,156.0,1.83,-0.46,-25.0,1.0,5.0
세방,47.94,0.32,240.0,0.68,0.52,55.0,2.0,6.0
KC코트렐,32.18,0.55,45.0,1.71,1.00,8.0,2.0,7.0
대양제지 *,0.00,0.37,279.0,1.64,1.03,25.0,3.0,10.0
제이엔케이히터 *,0.00,0.37,279.0,1.64,1.03,25.0,3.0,10.0
성광벤드 *,0.00,0.37,279.0,1.64,1.03,25.0,3.0,10.0
